In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print('mount success')

Mounted at /content/drive
mount success


In [ ]:
dataset_path = '/content/drive/Shareddrives/msvd-dataset'

# for training dataset
train_path = os.path.join(dataset_path, 'train')
output_path = '/content/drive/Shareddrives/msvd-train-feats/train/custom_feat'

# for validation dataset
#train_path = os.path.join(dataset_path, 'val')
#output_path = '/content/drive/Shareddrives/msvd-test-feats/val/custom_feat'

# for test dataset
#train_path = os.path.join(dataset_path, 'test')
#output_path = '/content/drive/Shareddrives/msvd-test-feats/test/custom_feat'

In [ ]:
video_folder = 'video'

## FEATURE EXTRACTION using VGG16 and YOLOv8 (for training)

In [ ]:
import shutil
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 10.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from ultralytics import YOLO

In [ ]:
#splitting video clip into frames
# returns list of frame names

def video_to_frames(video):
    path = os.path.join(train_path, 'temporary_images')
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    video_path = os.path.join(train_path, video_folder, video)
    count = 0
    image_list = []

    # Path to video file
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        cv2.imwrite(os.path.join(train_path, 'temporary_images', 'frame%d.jpg' % count), frame)
        image_list.append(os.path.join(train_path, 'temporary_images', 'frame%d.jpg' % count))
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    print('Frames extracted')
    return image_list

VGG16 CNN MODEL

In [ ]:
def model_cnn_load():
    model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final

In [ ]:
def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img

YOLOv8 OBJECT DETECTION MODEL

In [ ]:
yolo_model_name = "yolov8n.pt"
yolo_model = YOLO(yolo_model_name)

100%|██████████| 6.23M/6.23M [00:00<00:00, 111MB/s]


FEATURE EXTRACTION

80 frames per video are taken.
From each frame -> 4096 features are extracted using VGG16 and 30 detection features using YOLO.

The features are stacked to form an (80, 4096 + 30) shaped array.

In [ ]:
# extracting features for each video

def extract_features(video, model):
    """
    :param video: The video whose frames are to be extracted to convert into a numpy array
    :param model: the pretrained vgg16 model
    :return: numpy array of size 4096x80
    """

    video_id = video.split(".")[0]
    print(video_id)
    print(f'Processing video {video}')


    image_list = video_to_frames(video) #get frame list for the video
    samples = np.round(np.linspace(0, len(image_list) - 1, 80))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))

    yolo_img_feats = np.array([]) #contains (6*5) * 80 elements
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img

        #object detection
        results = yolo_model.predict(source=images[i], save=True) # single frame, many objects
        out_list = []
        out = results[0].boxes.data.tolist() # list of detected objects for 1 frame
        for r in out:
          if r[-1] == 0:
            r[-1] = 80
          out_list.append(r)

        while len(out_list) < 6:
          out_list.append([0, 0, 0, 0, 0, 0])

        #print(out_list)
        # sorting the list based on score and taking first 5 objects
        sorted_out_list = sorted(out_list, key=lambda x: x[4], reverse=True)[:5]
        #print(sorted_out_list)
        sorted_array = np.array(sorted_out_list).flatten() #1D array of obj detection info - each frame i (30)
        #print('sorted array', sorted_array)
        yolo_img_feats = np.append(yolo_img_feats, sorted_array)


    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats) # 80 frames' vgg features extracted (80,4096)


    # deleting the frame image files
    temp_images_dir = os.path.join(train_path, 'temporary_images')
    try:
      shutil.rmtree(temp_images_dir)
    except FileNotFoundError:
        print(f"Directory '{temp_images_dir}' not found.")
    return (img_feats, yolo_img_feats)

In [ ]:
# Saves the numpy features from all the videos. Passes videos one by one to extract_features() fn, and saves the received features into npy file

def extract_feats_pretrained_cnn():
    model = model_cnn_load()
    print('VGG16 model loaded')

    #for saving features
    if not os.path.isdir(os.path.join(output_path, 'feat')):
        os.mkdir(os.path.join(output_path, 'feat'))
    if not os.path.isdir(os.path.join(output_path, 'yolo-feat')):
        os.mkdir(os.path.join(output_path, 'yolo-feat'))

    video_list = os.listdir(os.path.join(train_path, video_folder))
    for video in video_list:
        video_name = video.split(".")[0]
        npy_file = os.path.join(output_path, 'feat', video_name + '.npy')
        yolo_npy_file = os.path.join(output_path, 'yolo-feat', video_name + '.npy')

        # Check if corresponding .npy file already exists
        if os.path.exists(npy_file) and os.path.exists(yolo_npy_file):
            print(f"Skipping {video_name} as features already exist.")
            continue

        # Extract features for the video and save into feat folder
        img_feats, yolo_img_feats = extract_features(video, model)
        np.save(npy_file, img_feats)
        np.save(yolo_npy_file, yolo_img_feats)

In [ ]:
if __name__ == "__main__":
  extract_feats_pretrained_cnn()

Streaming output truncated to the last 5000 lines.
Speed: 9.4ms preprocess, 8.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 (no detections), 9.8ms
Speed: 7.9ms preprocess, 9.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 1 vase, 9.4ms
Speed: 10.8ms preprocess, 9.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 (no detections), 8.6ms
Speed: 9.0ms preprocess, 8.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 1 person, 15.8ms
Speed: 10.3ms preprocess, 15.8ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 (no detections), 8.9ms
Speed: 11.6ms preprocess, 8.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/de